In [1]:
import numpy as np
import pandas as pd
from main import tree_grow, tree_grow_b, tree_pred, tree_pred_b
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from IPython.display import display, Markdown, Latex

In [2]:
train = pd.read_csv("eclipse-metrics-packages-2.0.csv", delimiter=";")
test = pd.read_csv("eclipse-metrics-packages-3.0.csv", delimiter=";")
list(train.columns[4:44])
feats = ['pre',
 'ACD_avg',
 'ACD_max',
 'ACD_sum',
 'FOUT_avg',
 'FOUT_max',
 'FOUT_sum',
 'MLOC_avg',
 'MLOC_max',
 'MLOC_sum',
 'NBD_avg',
 'NBD_max',
 'NBD_sum',
 'NOCU',
 'NOF_avg',
 'NOF_max',
 'NOF_sum',
 'NOI_avg',
 'NOI_max',
 'NOI_sum',
 'NOM_avg',
 'NOM_max',
 'NOM_sum',
 'NOT_avg',
 'NOT_max',
 'NOT_sum',
 'NSF_avg',
 'NSF_max',
 'NSF_sum',
 'NSM_avg',
 'NSM_max',
 'NSM_sum',
 'PAR_avg',
 'PAR_max',
 'PAR_sum',
 'TLOC_avg',
 'TLOC_max',
 'TLOC_sum',
 'VG_avg',
 'VG_max',
 'VG_sum']
train_x = train[feats]
train_y = train["post"]
test_x = test[feats]
test_y = test["post"]

In [3]:
train_x_np = train_x.to_numpy()
test_x_np = test_x.to_numpy()

In [4]:
train_y_np = train_y.to_numpy()
train_y_np = np.where(train_y_np > 0, 1, 0)
test_y_np = test_y.to_numpy()
test_y_np = np.where(test_y_np > 0, 1, 0)

In [5]:
single_tree = tree_grow(train_x_np, train_y_np, 15, 5, 41)
test_y_np_pred_st = tree_pred(test_x_np, single_tree)

In [6]:
# len(single_tree.y) == (len(single_tree.l.y) + len(single_tree.r.y))

In [7]:
def report_scores(y_true,y_pred):
    conf_mat = confusion_matrix(y_true,y_pred)
    tn = conf_mat[0][0]
    fp = conf_mat[0][1]
    fn = conf_mat[1][0]
    tp = conf_mat[1][1]
    print("Confusion matrix:")
    display(Markdown(f'''| True/Pred |Pos|Neg|
|-----------|---|---|
|       Pos |{tp}|{fn}|
|       Neg |{fp}|{tn}|
    '''))
    print("---------------------------------")
    print("LaTex formatting:", '''
\\begin{table}[]
\\begin{tabular}{|r|l|l|}
\\hline
True/Pred & Pos & Neg \\\ \hline''')
    print(f"Pos                             &  {tp}   &  {fn}   \\\ \hline")
    print(f"Neg                             & {fp}    &  {tn}   \\\ \hline")
    print('''\end{tabular}
\end{table}''')
    print("------------------------------------")
    print("accuracy:", accuracy_score(y_true,y_pred))
    print("precision:", precision_score(y_true,y_pred))
    print("recall:", recall_score(y_true,y_pred))

In [8]:
report_scores(test_y_np_pred_st, test_y_np)

Confusion matrix:


| True/Pred |Pos|Neg|
|-----------|---|---|
|       Pos |184|79|
|       Neg |129|269|
    

---------------------------------
LaTex formatting: 
\begin{table}[]
\begin{tabular}{|r|l|l|}
\hline
True/Pred & Pos & Neg \\ \hline
Pos                             &  184   &  79   \\ \hline
Neg                             & 129    &  269   \\ \hline
\end{tabular}
\end{table}
------------------------------------
accuracy: 0.6853252647503782
precision: 0.5878594249201278
recall: 0.6996197718631179


In [9]:
# Print the first two-layers of the tree
# Root node information
root = single_tree
split_feature = feats[root.f]
split_value = root.s
y_values = root.y
y_values_1_ratio = sum(y_values)/len(y_values)
print(f"Root values: split_feature = {split_feature}, split_value = {split_value}, y_values_1_ratio = {y_values_1_ratio}")

Root values: split_feature = pre, split_value = 4.5, y_values_1_ratio = 0.5039787798408488


In [10]:
print(np.bincount(single_tree.y))
# print(single_tree.f)
print(single_tree.s)
print(feats[single_tree.f])

[187 190]
4.5
pre


In [11]:
print(np.bincount(single_tree.l.y))
print(single_tree.l.s)
print(feats[single_tree.l.f])

[156  58]
26.5
VG_max


In [12]:
print(np.bincount(single_tree.r.y))
print(single_tree.r.s)
print(feats[single_tree.r.f])

[ 31 132]
0.1583333333333333
NOI_avg


In [13]:
bagging_tree = tree_grow_b(train_x_np, train_y_np, 15, 5, 41, 100)
test_y_np_pred_bt = tree_pred_b(test_x_np, bagging_tree)

In [15]:
report_scores(test_y_np_pred_bt, test_y_np)

Confusion matrix:


| True/Pred |Pos|Neg|
|-----------|---|---|
|       Pos |212|42|
|       Neg |101|306|
    

---------------------------------
LaTex formatting: 
\begin{table}[]
\begin{tabular}{|r|l|l|}
\hline
True/Pred & Pos & Neg \\ \hline
Pos                             &  212   &  42   \\ \hline
Neg                             & 101    &  306   \\ \hline
\end{tabular}
\end{table}
------------------------------------
accuracy: 0.783661119515885
precision: 0.6773162939297125
recall: 0.8346456692913385


In [16]:
random_tree = tree_grow_b(train_x_np, train_y_np, 15, 5, 6, 100)
test_y_np_pred_rt = tree_pred_b(test_x_np, random_tree)

In [18]:
report_scores(test_y_np_pred_rt, test_y_np)

Confusion matrix:


| True/Pred |Pos|Neg|
|-----------|---|---|
|       Pos |219|61|
|       Neg |94|287|
    

---------------------------------
LaTex formatting: 
\begin{table}[]
\begin{tabular}{|r|l|l|}
\hline
True/Pred & Pos & Neg \\ \hline
Pos                             &  219   &  61   \\ \hline
Neg                             & 94    &  287   \\ \hline
\end{tabular}
\end{table}
------------------------------------
accuracy: 0.7655068078668684
precision: 0.6996805111821086
recall: 0.7821428571428571
